In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from time import time

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GRU, Embedding, Dropout, Activation, LSTM
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
# import gensim.models.keyedvectors as word2vec
# import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score


/home/andy/anaconda3/envs/ml/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path = './data/'
EMBEDDING_FILE=f'{path}glove.6B.50d.txt'
TRAIN_DATA_FILE=f'{path}train.csv'
TEST_DATA_FILE=f'{path}test.csv'

In [3]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

print(f'Train shape: {train.shape}')
print(f'Test shape: {test.shape}')

Train shape: (159571, 8)
Test shape: (153164, 2)


In [5]:
train_sample = train.sample(frac=.5, random_state=29)

train_sample.shape

(79786, 8)

In [6]:
list_sentences_train = train_sample["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train_sample[list_classes].values

In [7]:
embed_size = [50, 100, 150, 200, 250, 300]
max_features = [10000, 20000, 30000, 40000, 50000, 60000]
max_len = [50, 100, 150, 200, 250, 300]

In [8]:
x_train, x_test, y_train, y_test = train_test_split(list_sentences_train, y, test_size=0.5, random_state=29)

In [9]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

In [10]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

(0.020940498, 0.6441043)

First run for LSTM

In [11]:
# Sould i also tune drop out?
resul = pd.DataFrame(columns=['layer','embed_size','max_features','max_len','time_taken','toxic','severe_toxic','obscene','threat','insult','identity_have','average'])
rown = 0
for es in embed_size:
    for mf in max_features:
        for ml in max_len:
            start = time()
            tokenizer = Tokenizer(num_words=mf)
            tokenizer.fit_on_texts(list(list_sentences_train))
            list_tokenized_train = tokenizer.texts_to_sequences(x_train)
            list_tokenized_test = tokenizer.texts_to_sequences(x_test)
            X_t = pad_sequences(list_tokenized_train, maxlen=ml)
            X_te = pad_sequences(list_tokenized_test, maxlen=ml)
            
            word_index = tokenizer.word_index
            nb_words = min(mf, len(word_index))
            embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, es))
            for word, i in word_index.items():
                if i >= mf: continue
                embedding_vector = embeddings_index.get(word)
                if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
            inp = Input(shape=(ml,))
            x = Embedding(mf, es, weights=[embedding_matrix])(inp)
            x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
            x = GlobalMaxPool1D()(x)
            x = Dense(50, activation="relu")(x)
            x = Dropout(0.1)(x)
            x = Dense(6, activation="sigmoid")(x)
            model = Model(inputs=inp, outputs=x)
            model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
            
            print(f'Fitting Embed Size: {es}, Max Features: {mf}, Max Len: {ml}\n')
            model.fit(X_t, y_train, batch_size=32, epochs=2)
            print('\n')
            print(f'Predicting Embed Size: {es}, Max Features: {mf}, Max Len: {ml}\n')
            y_pred = model.predict([X_te], batch_size=1024, verbose=1)
            print('\n')
            print(f'ROC AUC for Embed Size: {es}, Max Features: {mf}, Max Len: {ml}\n')
            
            tox_rocauc = roc_auc_score(y_test[:,0], y_pred[:,0])
            stox_rocauc = roc_auc_score(y_test[:,1], y_pred[:,1])
            obs_rocauc = roc_auc_score(y_test[:,2], y_pred[:,2])
            thr_rocauc = roc_auc_score(y_test[:,3], y_pred[:,3])
            ins_rocauc = roc_auc_score(y_test[:,4], y_pred[:,4])
            idh_rocauc = roc_auc_score(y_test[:,5], y_pred[:,5])
            avg_rocauc = (tox_rocauc+stox_rocauc+obs_rocauc+thr_rocauc+ins_rocauc+idh_rocauc)/6

            print(f'Toxic: {tox_rocauc}')
            print(f'S Tox: {stox_rocauc}')
            print(f'Obs:   {obs_rocauc}')
            print(f'Thr:   {thr_rocauc}')
            print(f'Ins:   {ins_rocauc}')
            print(f'IDH:   {idh_rocauc}')
            print(f'Avrg:  {avg_rocauc}')
            print('\n')
            
            end = time()
            
            resul.loc[rown] = ['LSTM', es, mf, ml, end-start, tox_rocauc, stox_rocauc, obs_rocauc, thr_rocauc, ins_rocauc, idh_rocauc, avg_rocauc]
            rown+=1
            

resul.to_csv('./results.csv', float_format='%.3f', index=False)

Fitting Embed Size: 50, Max Features: 10000, Max Len: 50

Epoch 1/2
39893/39893 [==============================] - 37s 921us/step - loss: 0.0807 - acc: 0.9742
Epoch 2/2
39893/39893 [==============================] - 38s 945us/step - loss: 0.0545 - acc: 0.9809


Predicting Embed Size: 50, Max Features: 10000, Max Len: 50

39893/39893 [==============================] - 4s 104us/step


ROC AUC for Embed Size: 50, Max Features: 10000, Max Len: 50

Toxic: 0.9666885934495053
S Tox: 0.9865199179689274
Obs:   0.9802491453810922
Thr:   0.9454469602423142
Ins:   0.9747056945467912
IDH:   0.9626996106630129
Avrg:  0.9693849870419405


Fitting Embed Size: 50, Max Features: 10000, Max Len: 100

Epoch 1/2
39893/39893 [==============================] - 69s 2ms/step - loss: 0.0844 - acc: 0.9729
Epoch 2/2
39893/39893 [==============================] - 69s 2ms/step - loss: 0.0522 - acc: 0.9812


Predicting Embed Size: 50, Max Features: 10000, Max Len: 100

39893/39893 [==============================] - 

Fitting Embed Size: 50, Max Features: 30000, Max Len: 100

Epoch 1/2
39893/39893 [==============================] - 76s 2ms/step - loss: 0.0833 - acc: 0.9731
Epoch 2/2
39893/39893 [==============================] - 74s 2ms/step - loss: 0.0516 - acc: 0.9814


Predicting Embed Size: 50, Max Features: 30000, Max Len: 100

39893/39893 [==============================] - 9s 216us/step


ROC AUC for Embed Size: 50, Max Features: 30000, Max Len: 100

Toxic: 0.9723686159633913
S Tox: 0.9886652425090141
Obs:   0.9852364498334162
Thr:   0.9599557737002428
Ins:   0.9792276740749034
IDH:   0.9668330914075898
Avrg:  0.9753811412480928


Fitting Embed Size: 50, Max Features: 30000, Max Len: 150

Epoch 1/2
39893/39893 [==============================] - 108s 3ms/step - loss: 0.0794 - acc: 0.9743
Epoch 2/2
39893/39893 [==============================] - 105s 3ms/step - loss: 0.0501 - acc: 0.9817


Predicting Embed Size: 50, Max Features: 30000, Max Len: 150

39893/39893 [==============================] -

Fitting Embed Size: 50, Max Features: 50000, Max Len: 150

Epoch 1/2
39893/39893 [==============================] - 112s 3ms/step - loss: 0.0815 - acc: 0.9739
Epoch 2/2
39893/39893 [==============================] - 109s 3ms/step - loss: 0.0503 - acc: 0.9816


Predicting Embed Size: 50, Max Features: 50000, Max Len: 150

39893/39893 [==============================] - 13s 332us/step


ROC AUC for Embed Size: 50, Max Features: 50000, Max Len: 150

Toxic: 0.9745398454739556
S Tox: 0.9888362893873384
Obs:   0.9859848847160406
Thr:   0.9547668868741878
Ins:   0.9816516003588424
IDH:   0.9668044798089812
Avrg:  0.9754306644365576


Fitting Embed Size: 50, Max Features: 50000, Max Len: 200

Epoch 1/2
39893/39893 [==============================] - 142s 4ms/step - loss: 0.0791 - acc: 0.9739
Epoch 2/2
39893/39893 [==============================] - 139s 3ms/step - loss: 0.0502 - acc: 0.9814


Predicting Embed Size: 50, Max Features: 50000, Max Len: 200

39893/39893 [==============================

ValueError: could not broadcast input array from shape (50) into shape (100)

In [12]:
resul.head
resul.to_csv('./results.csv', float_format='%.3f', index=False)